# Census LEHD Origin Destination analysis
Goal: determine which census tracts have the highest number of jobs that people commute to from another county.

In [1]:
import pandas as pd
import numpy as np
import sys, os

In [2]:
# load origin destination data for california 2015
dirname = os.path.dirname(os.path.realpath("__file__"))
data = pd.read_csv("data/ca_od_main_JT00_2015.csv.gz", compression="gzip")

In [3]:
# load crosswalk table to join by
cxwalk = pd.read_csv("data/ca_xwalk.csv.gz", compression="gzip", encoding="ISO-8859-1", low_memory=False)

In [4]:
# greater bay area counties
cty_fips_list = [6001, 6013, 6041, 6047, 6055, 6067, 6069, 6075, 6077, 6081, 6085, 6087, 6095, 6097, 6099]

In [5]:
# sf bay area counties
target_cty_fips_list = [6001, 6013, 6041, 6055, 6075, 6081, 6085, 6095, 6097]

In [6]:
# filter rows in crosswalk to include only SF Bay counties and fields we need
cxwalk = cxwalk[cxwalk['cty'].isin(cty_fips_list)]
cxwalk = cxwalk[['trct', 'tabblk2010', 'cty', 'ctyname']]

In [7]:
# perform an inner join on crosswalk data and origin destination data to keep only blocks for SF Bay Area
data_h = data.merge(cxwalk, how="inner", left_on="h_geocode", right_on="tabblk2010")

In [8]:
# create seperate home and work county fips columns as strings to compare with one another
data_h['cty_h'] = data_h['h_geocode'].apply(str).str.slice(0,4)
data_h['cty_w'] = data_h['w_geocode'].apply(str).str.slice(0,4)
data_h['cty_h'] = pd.to_numeric(data_h['cty_h'], errors='coerce')
data_h['cty_w'] = pd.to_numeric(data_h['cty_w'], errors='coerce')

In [9]:
# inspect data
data_h
# data_h.dtypes

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,trct,tabblk2010,cty,ctyname,cty_h,cty_w
0,60014001001007,60014003004007,1,0,1,0,0,0,1,0,0,1,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
1,60014003003017,60014003004007,1,0,0,1,0,1,0,0,1,0,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
2,60014003003018,60014003004007,1,1,0,0,0,1,0,1,0,0,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
3,60014003004011,60014003004007,1,0,1,0,1,0,0,0,0,1,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
4,60014010001002,60014003004007,1,1,0,0,0,0,1,0,0,1,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
5,60014011004018,60014003004007,1,1,0,0,1,0,0,0,0,1,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
6,60014028001007,60014003004007,1,1,0,0,0,1,0,0,0,1,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
7,60014028002035,60014003004007,1,0,0,1,0,0,1,0,0,1,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
8,60014030001002,60014003004007,1,0,0,1,0,0,1,0,1,0,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001
9,60014033001008,60014003004007,1,0,1,0,0,1,0,0,0,1,20170919,6001400300,60014003004007,6001,"Alameda County, CA",6001,6001


In [10]:
# aggregate blocks to tracts while computing the number of local and away jobs per tract
data_agg = data_h.groupby(['trct', 'cty'], as_index=False).apply(lambda x: pd.Series(dict(
    sum_local=(x.cty_h == x.cty_w).sum(),
    sum_away=((x.cty_h != x.cty_w) & x.cty_w.isin(target_cty_fips_list)).sum() # only counting away as different from home county and within the 9 county bay area
)))

In [11]:
# compute proportion of away vs local
data_agg['ratio_away_to_local'] = data_agg['sum_away'] / data_agg['sum_local']

In [12]:
# sort by ratio column descending
data_agg = data_agg.sort_values('ratio_away_to_local', ascending=False)

In [13]:
# inspect the data
data_agg

,,sum_local,sum_away,ratio_away_to_local
trct,cty,,,
6095253000,6095,27,634,23.481481
6041122000,6041,21,388,18.476190
6041990100,6041,12,141,11.750000
6055200900,6055,23,193,8.391304
6081990100,6081,23,137,5.956522
6013389100,6013,154,842,5.467532
6087100400,6087,132,719,5.446970
6095252802,6095,42,215,5.119048
6013389200,6013,152,750,4.934211


In [14]:
# write processed data to csv file
outfile = os.path.join(dirname, 'data/lehd_od_tracts_home_ratio_gba.csv')
data_agg.to_csv(outfile, encoding="utf-8")